# Spatial LDA Preprocessing

## Overview and Setup
This notebook walks through the various steps of transforming and evaluating a standard cell table to be compatible with Spatial-LDA analysis.  The main topics covered are:

- [Correctly Formatting a Cell Table](#Correctly-Formatting-a-Cell-Table)
- [Defining Local Cellular Neighborhoods](#Defining-Local-Cellular-Neighborhoods)
- [Calculating Neighborhood Characteristics](#Calculating-Neighborhood-Characteristics)
- [Determining the Number of Topics](#Determining-the-Number-of-Topics)
- [Saving Results](#Saving-Results)

#### Import Required Packages

In [1]:
import os

import pandas as pd

import ark.settings as settings
from ark.utils import spatial_lda_utils
from ark.analysis import visualize
from ark.spLDA import processing


#### Set Up File Paths
- `base_dir`: working directory
- `input_dir`: contains the standard cell table
- `cell_table_path`: name of the file for the standard cell table
- `processed_dir`: destination directory for the processed data
- `viz_dir`: destination directory for all plots and visualizations

In [8]:
base_dir = "../data/example_dataset"
input_dir = os.path.join(base_dir, "spatial_enrichment_input_data")
cell_table_path = os.path.join(input_dir, "example_expression_matrix_clustered.csv")
processed_dir = os.path.join(base_dir, "spatial_lda_processed_data")
viz_dir = os.path.join(base_dir, "spatial_lda_visualizations")

# Create directories if they do not exist
for directory in [input_dir, processed_dir, viz_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

## Correctly Formatting a Cell Table
In order to proceed, you'll need data in the form of a standard cell table similar to the output of __[generate_cell_table()](https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.segmentation.html#ark.segmentation.marker_quantification.generate_cell_table)__.  The cell table should be in `.csv` format.

Assuming the data have undergone pixel clustering, the cell table should contain the following columns in addition to any marker-specific columns:

In [2]:
print(settings.BASE_COLS)

['SampleID', 'label', 'cell_size', 'centroid-0', 'centroid-1', 'FlowSOM_ID', 'cluster_labels']


#### Import and Inspect the Cell Table

In [14]:
cell_table = pd.read_csv(cell_table_path)
print(cell_table.columns)

Index(['cell_size', 'Background', 'BetaCatenin', 'BetaTubulin', 'CD20', 'CD3',
       'CD4', 'CD45', 'CD8', 'CD9', 'ECadherin', 'ER', 'GLUT1', 'HER2', 'HH3',
       'HLA_Class_1', 'Ki67', 'LaminAC', 'Membrane', 'NaK ATPase',
       'PanKeratin', 'SMA', 'Vimentin', 'label', 'area', 'eccentricity',
       'major_axis_length', 'minor_axis_length', 'perimeter', 'convex_area',
       'equivalent_diameter', 'centroid-0', 'centroid-1',
       'major_minor_axis_ratio', 'perim_square_over_area',
       'major_axis_equiv_diam_ratio', 'convex_hull_resid', 'centroid_dif',
       'num_concavities', 'fov'],
      dtype='object')
(566, 40)
(3262, 53)


In [4]:
cell_table.head()

,Unnamed: 0,cell_size,C,Na,Si,HH3,Vimentin,SMA,Background,Collagen.1,...,label,SampleID,Tissue,PatientID,lineage,cell_type,cell_lin,lintype_num,FlowSOM_ID,cluster_labels
0,0,169,0.911187,0.813581,0.287503,0.765556,0.786841,0.0,0.098785,0.000000,...,2,fov6,gran_lung,30,endothelial,endothelial,nonimmune,5,1,4
1,1,240,0.912404,0.857459,0.287738,0.881312,0.726815,0.0,0.291964,0.018747,...,3,fov6,gran_lung,30,endothelial,endothelial,nonimmune,5,1,0
2,2,40,0.890122,0.838072,0.302053,0.839182,0.795838,0.0,0.202693,0.000000,...,4,fov6,gran_lung,30,immune,CD16_CD14_Mono,myeloid,1,2,0
3,3,66,0.900242,0.861314,0.237600,0.797691,0.509005,0.0,0.241783,0.000000,...,5,fov6,gran_lung,30,immune,CD8_T,lymphocyte,2,3,4
4,4,386,0.891710,0.880452,0.249988,0.910139,0.544552,0.0,0.283618,0.000000,...,6,fov6,gran_lung,30,immune,CD4_T,lymphocyte,2,4,4


#### Format the Cell Table
The following code will transform the cell table into a format suitable for Spatial-LDA.  Here is where you can specify which clusters and/or markers to include in your analysis.

Indicate markers by their column name and clusters by their cluster ID (`FlowSOM_ID`).  If you do not want to include any markers, set `markers = None`, although you must specify at least one of clusters or markers.


In [6]:
# Specify which markers and clusters to include
markers = [
    "CD3",
    "CD4",
    "CD8",
    "CD14",
    "CD16",
    "CD20",
    "CD31",
    "CD36",
    "IDO",
    "PD.L1",
    "Ki67",
    "Vimetin",
    "Lag3"
]
clusters = [
    1,
    2,
    3,
    4,
    5,
    6,
    7,
    8,
    9,
    10,
    11,
    12,
    13,
    14,
    15,
    16,
    17,
    18,
    19,
    20
]

In [7]:
# Call formatting function
formatted_cell_table = processing.format_cell_table(
    cell_table=cell_table, markers=markers, clusters=clusters)

ValueError: Invalid value(s) provided for required columns variable: value(s) centroid-0,centroid-1 not found in cell table columns list

The result is now a dictionary with one element per field of view (FOV) which contains only the necessary columns for spatial-LDA.

## Defining Local Cellular Neighborhoods
Spatial-LDA tries to learn information about cells in similar "neighborhoods".  To do that, we need to define the size and location of each neighborhood.

Determining a reasonable neighborhood size around a particular cell will depend on the size and quantity of all cells in the FOV, but a typical size is a radius between 100-200 pixels.  The neighborhood around that cell then includes all cells whose centroid falls within the provided radius.

The code below collects metrics regarding the distribution of cell counts and cell size among all FOVs.  If the pixel dimension of your FOVs is anything other than 1,024 x 1,024 then indicate it using the `total_pix` argument.

In [ ]:
fov_stats = processing.fov_density(
    cell_table=formatted_cell_table, total_pix=1024 ** 2)

The output contains three metrics per FOV:
- `average_area`: average cell size (in pixel area)
- `cellular_density`: ratio of total pixels occupied by cells to total pixels
- `total_cells`: total number of individual cells

These metrics can be visualized below.

In [ ]:
# Average area
visualize.visualize_fov_stats(
    fov_stats, metric="average_area", save_dir=viz_dir)

In [ ]:
# Cellular density
visualize.visualize_fov_stats(
    fov_stats, metric="cellular_density", save_dir=viz_dir)

In [ ]:
# Total cells
visualize.visualize_fov_stats(
    fov_stats, metric="total_cells", save_dir=viz_dir)

#### Neighborhood Size
If a large number of FOVs have low cell counts, then defining a larger neighborhood size in the next section may be warranted.  On the other hand, if many of the FOVs are densely packed with cells then a smaller neighborhood size will likely perform better.

## Calculating Neighborhood Characteristics
The next steps combine the radius information learned above with marker or cluster data to construct cellular neighborhoods and measure adjacent cells.

#### Featurize the Cell Table
In this step, the neighborhood around each index cell is summarized using one of four featurization methods.  The four methods are:

- `cluster`: for each cluster, count the total number of cells within the neighborhood of the index cell which belong to that cluster.
- `marker`: for each marker, count the total number of cells within the neighborhood of the index cell which express that marker.
- `avg_marker`: for each marker, compute the average expression level of all cells within the neighborhood of the index cell.
- `count`: count the total number of cells within the neighborhood of the index cell.

By default, all cells are labelled as index cells, but you can choose to only use specific cells if desired.  If so, you must create a new column to indicate which cells are index cells.  Below is an example of code that would create an index column for tumor cells:

In [ ]:
# Example: create custom index column for tumor cells

# for i in formatted_cell_table["fovs"]:
#     is_tumor = formatted_cell_table[i]["cluster"] == "tumor"
#     formatted_cell_table[i]["is_tumor"] = is_tumor

In [ ]:
# Set featurization parameters
featurization = "cluster"
radius = 100
cell_index = "is_index"

In [ ]:
# Call featurization function
featurized_cell_table = processing.featurize_cell_table(
    cell_table=formatted_cell_table, featurization=featurization,
    radius=radius, cell_index=cell_index)

In addition to summarizing the cellular neighborhoods, `featurize_cell_table()` will also set aside a fraction of the data to use as a training set.

#### Constructing the Adjacency Network
The following code computes pairwise distances between cells and uses this information to build a network graph of adjacent cells.  Cells are considered adjacent if they share a facet in the Voronoi partitioning of cell positions.  Spatial-LDA uses this information to regulate how likely adjacent cells are to have similar topic preferences.

In [ ]:
difference_mats = processing.create_difference_matrices(
    cell_table=formatted_cell_table, features=featurized_cell_table)

The output contains the difference matrices for the training data and also the full combined data.

In [ ]:
difference_mats["train_diff_mat"].head()

## Determining the Number of Topics
Training a spatial-LDA model can be computationally expensive, so it's worth exploring some reasonable values of topic parameters before running the algorithm.  Five different metrics are currently supported for evaluating a K-means clustering of the featurized cell table where the number of K-means clusters is a proxy for the number of topics.

The five different metrics are:

- `inertia`: Inertia is the total sum of within-cluster variance for all clusters.
- `silhouette`: The silhouette score is a goodness-of-fit measurement for clustering.  Values closer to 1 indicate that most observations are well-matched to their cluster, while values closer to -1 indicate poorly matched observations.
- `gap_stat`: The gap statistic, $Gap(k)$, is a re-sampling based measure which computes the difference between the log of the pooled within-cluster sum of squares ( $logW_k$ ) to its expected value ( $E(logW_k)$ ) under a null distribution.  The optimal number of clusters $k$ is the smallest $k$ for which $Gap(k) \geq Gap(k+1) - s_{k+1}$ where $s_{k+1}$ is a scaled estimate of the standard error of $Gap(k+1)$.
- `percent_var_exp`: The percent of total variance in the data explained by the clustering.
- `cell_counts`: The distribution of cell features within each cluster.

#### Computing EDA Metrics
The code block below allows you to specify different number of topics to explore using all five metrics.

In [ ]:
# specify different topic numbers and bootstrap iterations
num_topics = [3, 4, 5, 6, 7, 8, 9, 10]
num_boots = 100

In [ ]:
topic_eda = processing.compute_topic_eda(
    features=featurized_cell_table["featurized_fovs"],
    topics=num_topics,
    num_boots=num_boots)

#### Visualizing EDA Metrics

The following code blocks produce plots of each metric as a function of the number of clusters (topics).

In [ ]:
# Inertia
visualize.visualize_topic_eda(
    data=topic_eda, metric="inertia", save_dir=viz_dir)

In [ ]:
# Silhouette Score
visualize.visualize_topic_eda(
    data=topic_eda, metric="silhouette", save_dir=viz_dir)

In [ ]:
# Gap Statistic
visualize.visualize_topic_eda(
    data=topic_eda, metric="gap_stat", save_dir=viz_dir)

In [ ]:
# Percent of Variance Explained
visualize.visualize_topic_eda(
    data=topic_eda, metric="percent_var_exp", save_dir=viz_dir)

The code below plots a heatmap of the cell features against the specific cluster assignments from a given K-means clustering.  You must specify one value of `k` present in the `num_topics` variable above.

In [ ]:
# Cell Feature Distribution
k=num_topics[0]
visualize.visualize_topic_eda(
    data=topic_eda, metric="cell_counts", k=k, save_dir=viz_dir)

## Saving Results
The plots and figures above will be saved in the directory specified by `viz_dir` if it was passed to the visualization functions.  To save the formatted and/or featurized cell tables along with the difference matrices, use the code block below.

*Note: entire dictionary objects can be saved into .pkl files.  If you want to save data frames for specific FOVs individually as .csv files, you need to extract the desired data frame and specify format="csv" in the saving function.*

In [ ]:
# save formatted cell table
file_name = "formatted_cell_table"
spatial_lda_utils.save_spatial_lda_file(
    data=formatted_cell_table, dir=processed_dir, file_name=file_name, format="pkl")

In [ ]:
# save featurized cell table
file_name = "featurized_cell_table"
spatial_lda_utils.save_spatial_lda_file(
    data=featurized_cell_table, dir=processed_dir, file_name=file_name, format="pkl")

In [ ]:
# save difference matrices
file_name = "difference_mats"
spatial_lda_utils.save_spatial_lda_file(
    data=difference_mats, dir=processed_dir, file_name=file_name, format="pkl")

In [ ]:
# save FOV stats
file_name = "fov_stats"
spatial_lda_utils.save_spatial_lda_file(
    data=fov_stats, dir=processed_dir, file_name=file_name, format="pkl")

In [ ]:
# save topic EDA
file_name = "topic_eda"
spatial_lda_utils.save_spatial_lda_file(
    data=topic_eda, dir=processed_dir, file_name=file_name, format="pkl")